<h1 align='center'>Алгоритмы анализа данных</h1>

<h1 align='center'>Домашнее задание № 4</h1>

<h2 align='left'>Деревья решений</h2>

# <p style="background-color:lightgreen;font-family:newtimeroman;color:#662e2e;font-size:130%;text-align:center;border-radius:30px 30px;">Задача 1</p>

__1.__ Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

__Решение:__

In [1]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
data, labels = make_regression(n_samples=1000, n_features=2, n_informative=2, n_targets=1, random_state=42)

In [3]:
class Node:

    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [4]:
class Leaf:

    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()

    def predict(self):
        objects = []  # сформируем список объектов в листе
        for label in self.labels:
            objects.append(label)
        #  найдем среднее
        objects = np.array(objects)
        prediction = objects.mean()
        return prediction

In [5]:
def means(labels):
    objects = []
    for label in labels:
        objects.append(label)
    objects = np.array(objects)
    prediction= objects.mean()
    return np.mean((labels - prediction) ** 2)

In [6]:
# Расчет прироста

def gain(left_labels, right_labels, current_mean):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_labels.shape[0] / (left_labels.shape[0] + right_labels.shape[0]))

    return current_mean - p * means(left_labels) - (1 - p) * means(right_labels)

In [7]:
# Разбиение датасета в узле

def split(data, labels, index, t):

    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)

    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]

    return true_data, false_data, true_labels, false_labels

In [8]:
def find_best_split(data, labels):

    #  обозначим минимальное количество объектов в узле
    min_leaf = 5

    current_mean = means(labels)

    best_gain = 0
    best_t = None
    best_index = None

    n_features = data.shape[1]

    for index in range(n_features):
        t_values = [row[index] for row in data]

        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue

            current_gain = gain(true_labels, false_labels, current_mean)

            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [9]:
def build_tree(data, labels):

    gain, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [10]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [11]:
def predict(data, tree):

    predictions = []
    for obj in data:
        prediction = classify_object(obj, tree)
        predictions.append(prediction)
    return predictions

In [12]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = 0.3, random_state = 1)

In [13]:
my_tree = build_tree(train_data, train_labels)

In [14]:
# Напечатаем ход нашего дерева

def print_tree(node, spacing=''):
    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + 'Прогноз:', node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index), '<=', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + '  ')

    # Рекурсионный вызов функции на отрицательном поддереве
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

print_tree(my_tree)

Индекс 1 <= -0.09858813228426173
--> True:
  Индекс 1 <= -0.9824873935383096
  --> True:
    Индекс 1 <= -1.801057734152739
    --> True:
      Индекс 1 <= -2.151815401329305
      --> True:
        Прогноз: -98.46277114648234
      --> False:
        Индекс 0 <= 0.18645431476942764
        --> True:
          Индекс 1 <= -2.025142586657607
          --> True:
            Прогноз: -87.18539957448927
          --> False:
            Прогноз: -80.18576651420238
        --> False:
          Прогноз: -73.10590088904706
    --> False:
      Индекс 1 <= -1.2899608997410539
      --> True:
        Индекс 0 <= 0.543360192379935
        --> True:
          Индекс 1 <= -1.448013900416244
          --> True:
            Индекс 0 <= -0.09833962679474989
            --> True:
              Прогноз: -69.62529384187478
            --> False:
              Прогноз: -62.99840659606071
          --> False:
            Прогноз: -62.28234319105414
        --> False:
          Индекс 1 <= -1.55662917352390

In [15]:
# Получим ответы для обучающей выборки
train_answers = predict(train_data, my_tree)

In [16]:
# И получим ответы для тестовой выборки
answers = predict(test_data, my_tree)

In [17]:
def mse_(labels, res):
    return np.mean((labels - res) ** 2)

In [18]:
train_mse = mse_(train_labels, train_answers)
train_mse

5.491969205319535

In [19]:
test_mse = mse_(test_labels, answers)
test_mse

26.046179994641875

In [20]:
# Проверим точность модели с помощью метрики R2

def r_2(labels, res):
    return (1 - np.sum((res - labels) ** 2) / np.sum((labels - np.mean(labels))**2))

In [21]:
r_2(train_labels, train_answers)

0.9962795640198564

In [22]:
r_2(test_labels, answers)

0.985596370864998